# Inference with NLI validation

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re

In [2]:
prefix = "https://huggingface.co/muhammadravi251001/fine-tuned-FilteringNLI-"
suffix = "/blob/main/results/evaluation/metric_result.txt"

In [3]:
def em_and_f1(p_url, acc, id="L1", rounded_to_num=5):

    page = requests.get(p_url)
    soup = BeautifulSoup(page.content)

    tr = soup.find('tr', attrs={'id': id})
    td = tr.find(attrs={'class': 'overflow-visible whitespace-pre px-3'}).text
    x = re.findall("\d+\.\d+", td)

    if acc == True: return round(float(x[0]), rounded_to_num)
    else: return round(float(x[1]), rounded_to_num)

In [4]:
from collections import OrderedDict

def take_all_url(data):

    baseline = f"{prefix}indonli_mnli-{data}-TQ2-TS4-MS3-VA0-TH0.0{suffix}"
    msc_arr = ["indonli", "indonli_mnli", f"indonli_mnli_{data}-nli"]
    tq_arr = [1, 2]
    msi_arr = [1, 2, 3]
    var_arr = [1, 2, 3]
    th_arr = [0.25, 0.5, 0.75]

    url_arr = []
    url_arr.append(baseline)

    for msc in msc_arr:
        for tq in tq_arr:
            for msi in msi_arr:
                for var in var_arr:
                    for th in th_arr:
                        
                        if msc == "indonli" or msc == "indonli_mnli":
                            var = 1
                            th = 0.0

                        if var == 1:
                            th = 0.0

                        url_arr.append(f"{prefix}{msc}-{data}-TQ{tq}-TS4-MS{msi}-VA{var}-TH{th}{suffix}")
    
    url_arr = list(OrderedDict.fromkeys(url_arr))
    print("Amount of url:", len(url_arr))
    return url_arr

In [5]:
def extract_values(url, data):

    msc_list = f"indonli_mnli_{data}-nli|indonli_mnli|indonli"
    pattern = re.compile(r'.*FilteringNLI-({})-{}-TQ(\d+)-TS(\d+)-MS([\d]+)-VA([\d]+)-TH([\d.]+).*'.format(msc_list, data))
    match = pattern.match(url)
    
    if match:
        msc = match.group(1)
        TQ = int(match.group(2))
        TS = int(match.group(3))
        MSI = int(match.group(4))
        VA = int(match.group(5))
        
        TH = float(match.group(6))
        if TH == '0.00':
            TH = '0.0'
        elif TH == '0.50':
            TH = '0.5'

        EM = em_and_f1(url, True)
        F1 = em_and_f1(url, False)

        return msc, data, TQ, TS, MSI, VA, TH, EM, F1
    
    else:
        return None

In [6]:
import pandas as pd
import re
from tqdm import tqdm

def create_table(url_list, data):
    
    data_list = []
    for url in tqdm(url_list, desc="Processing URLs", unit="URL"):
        extracted_values = extract_values(url, data)
        data_list.append(extracted_values)

    df = pd.DataFrame(data_list, columns=['MSC', 'data', 'TQ', 'TS', 'MSI', 'VA', 'TH', 'EM', 'F1'])
    
    df.loc[0, 'MSC'] = "Baseline"
    df.loc[0, 'TQ'] = "BL"
    df.loc[0, 'TS'] = "BL"
    df.loc[0, 'MSI'] = "BL"
    df.loc[0, 'VA'] = "BL"
    df.loc[0, 'TH'] = "BL"
    
    return df

# IDK-MRC

In [7]:
data = "idkmrc"
url_list = take_all_url(data)
table = pd.DataFrame(create_table(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 55


Processing URLs: 100%|█████████████████████████████████████████████████████████████████| 55/55 [00:47<00:00,  1.16URL/s]


MSC    data  TQ  TS MSI  VA    TH        EM        F1
0                  Baseline  idkmrc  BL  BL  BL  BL    BL  75.82938  84.25128
1                   indonli  idkmrc   1   4   1   1   0.0  68.48341  75.02483
2                   indonli  idkmrc   1   4   2   1   0.0  54.62085  62.08946
3                   indonli  idkmrc   1   4   3   1   0.0  43.95735  51.95651
4                   indonli  idkmrc   2   4   1   1   0.0  75.35545  83.42190
5                   indonli  idkmrc   2   4   2   1   0.0  73.93365  82.22194
6                   indonli  idkmrc   2   4   3   1   0.0  72.63033  80.93617
7              indonli_mnli  idkmrc   1   4   1   1   0.0  71.32701  78.02263
8              indonli_mnli  idkmrc   1   4   2   1   0.0  61.96682  69.50875
9              indonli_mnli  idkmrc   1   4   3   1   0.0  52.60664  60.54560
10             indonli_mnli  idkmrc   2   4   1   1   0.0  75.47393  83.89583
11             indonli_mnli  idkmrc   2   4   2   1   0.0  72.63033  81.10442
12             indonli_mnli  idkmrc   2   4   3   1   0.0  71.44550  79.91959
13  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   1   0.0  75.82938  84.16834
14  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2  0.25  75.82938  84.16834
15  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2   0.5  75.82938  84.16834
16  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2  0.75  75.94787  84.16834
17  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3  0.25  75.82938  84.25128
18  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3   0.5  75.82938  84.25128
19  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3  0.75  75.82938  84.25128
20  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   1   0.0  61.49289  70.76557
21  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2  0.25  61.49289  70.76557
22  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2   0.5  60.90047  70.17316
23  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2  0.75  57.46445  66.69764
24  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3  0.25  59.95261  69.22529
25  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3   0.5  59.24171  68.51439
26  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3  0.75  54.26540  63.52821
27  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   1   0.0  56.75355  66.08548
28  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   2  0.25  56.75355  66.08548
29  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   2   0.5  56.04265  65.37458
30  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   2  0.75  51.65877  60.89986
31  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   3  0.25  54.62085  63.95278
32  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   3   0.5  53.79147  63.12339
33  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   3  0.75  47.39336  56.72332
34  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   1   0.0  75.82938  84.16834
35  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2  0.25  75.82938  84.16834
36  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2   0.5  75.82938  84.16834
37  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2  0.75  75.94787  84.16834
38  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3  0.25  75.82938  84.25128
39  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3   0.5  75.82938  84.25128
40  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3  0.75  75.82938  84.25128
41  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   1   0.0  62.44076  71.71344
42  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2  0.25  62.44076  71.71344
43  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2   0.5  61.72986  71.00254
44  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2  0.75  57.93839  67.17158
45  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3  0.25  61.13744  70.41012
46  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3   0.5  60.30806  69.58074
47  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3  0.75  54.97630  64.23911
48  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   1   0.0  57.81991  67.15183
49  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   2  0.25  57.81991  67.15183
50  indonli_mnli_idkmrc-nli  idkmrc   2   4   

In [8]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

idkmrc sorted by EM


MSC    data  TQ  TS MSI  VA    TH        EM        F1
37  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2  0.75  75.94787  84.16834
16  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2  0.75  75.94787  84.16834
0                  Baseline  idkmrc  BL  BL  BL  BL    BL  75.82938  84.25128
13  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   1   0.0  75.82938  84.16834
36  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2   0.5  75.82938  84.16834
35  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2  0.25  75.82938  84.16834
34  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   1   0.0  75.82938  84.16834
40  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3  0.75  75.82938  84.25128
39  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3   0.5  75.82938  84.25128
18  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3   0.5  75.82938  84.25128
17  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3  0.25  75.82938  84.25128
15  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2   0.5  75.82938  84.16834
14  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2  0.25  75.82938  84.16834
19  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3  0.75  75.82938  84.25128
38  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3  0.25  75.82938  84.25128
10             indonli_mnli  idkmrc   2   4   1   1   0.0  75.47393  83.89583
4                   indonli  idkmrc   2   4   1   1   0.0  75.35545  83.42190
5                   indonli  idkmrc   2   4   2   1   0.0  73.93365  82.22194
11             indonli_mnli  idkmrc   2   4   2   1   0.0  72.63033  81.10442
6                   indonli  idkmrc   2   4   3   1   0.0  72.63033  80.93617
12             indonli_mnli  idkmrc   2   4   3   1   0.0  71.44550  79.91959
7              indonli_mnli  idkmrc   1   4   1   1   0.0  71.32701  78.02263
1                   indonli  idkmrc   1   4   1   1   0.0  68.48341  75.02483
41  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   1   0.0  62.44076  71.71344
42  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2  0.25  62.44076  71.71344
8              indonli_mnli  idkmrc   1   4   2   1   0.0  61.96682  69.50875
43  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2   0.5  61.72986  71.00254
21  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2  0.25  61.49289  70.76557
20  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   1   0.0  61.49289  70.76557
45  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3  0.25  61.13744  70.41012
22  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2   0.5  60.90047  70.17316
46  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3   0.5  60.30806  69.58074
24  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3  0.25  59.95261  69.22529
25  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3   0.5  59.24171  68.51439
44  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2  0.75  57.93839  67.17158
48  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   1   0.0  57.81991  67.15183
49  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   2  0.25  57.81991  67.15183
23  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2  0.75  57.46445  66.69764
50  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   2   0.5  56.99052  66.32245
27  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   1   0.0  56.75355  66.08548
28  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   2  0.25  56.75355  66.08548
29  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   2   0.5  56.04265  65.37458
52  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   3  0.25  55.92417  65.25610
47  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3  0.75  54.97630  64.23911
53  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   3   0.5  54.97630  64.30823
31  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   3  0.25  54.62085  63.95278
2                   indonli  idkmrc   1   4   2   1   0.0  54.62085  62.08946
26  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3  0.75  54.26540  63.52821
32  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   3   0.5  53.79147  63.12339
9              indonli_mnli  idkmrc   1   4   3   1   0.0  52.60664  60.54560
51  indonli_mnli_idkmrc-nli  idkmrc   2   4   

In [9]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

idkmrc sorted by F1


MSC    data  TQ  TS MSI  VA    TH        EM        F1
0                  Baseline  idkmrc  BL  BL  BL  BL    BL  75.82938  84.25128
17  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3  0.25  75.82938  84.25128
40  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3  0.75  75.82938  84.25128
39  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3   0.5  75.82938  84.25128
18  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3   0.5  75.82938  84.25128
19  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   3  0.75  75.82938  84.25128
38  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   3  0.25  75.82938  84.25128
34  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   1   0.0  75.82938  84.16834
37  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2  0.75  75.94787  84.16834
36  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2   0.5  75.82938  84.16834
35  indonli_mnli_idkmrc-nli  idkmrc   2   4   1   2  0.25  75.82938  84.16834
13  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   1   0.0  75.82938  84.16834
14  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2  0.25  75.82938  84.16834
15  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2   0.5  75.82938  84.16834
16  indonli_mnli_idkmrc-nli  idkmrc   1   4   1   2  0.75  75.94787  84.16834
10             indonli_mnli  idkmrc   2   4   1   1   0.0  75.47393  83.89583
4                   indonli  idkmrc   2   4   1   1   0.0  75.35545  83.42190
5                   indonli  idkmrc   2   4   2   1   0.0  73.93365  82.22194
11             indonli_mnli  idkmrc   2   4   2   1   0.0  72.63033  81.10442
6                   indonli  idkmrc   2   4   3   1   0.0  72.63033  80.93617
12             indonli_mnli  idkmrc   2   4   3   1   0.0  71.44550  79.91959
7              indonli_mnli  idkmrc   1   4   1   1   0.0  71.32701  78.02263
1                   indonli  idkmrc   1   4   1   1   0.0  68.48341  75.02483
41  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   1   0.0  62.44076  71.71344
42  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2  0.25  62.44076  71.71344
43  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2   0.5  61.72986  71.00254
20  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   1   0.0  61.49289  70.76557
21  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2  0.25  61.49289  70.76557
45  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3  0.25  61.13744  70.41012
22  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2   0.5  60.90047  70.17316
46  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3   0.5  60.30806  69.58074
8              indonli_mnli  idkmrc   1   4   2   1   0.0  61.96682  69.50875
24  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3  0.25  59.95261  69.22529
25  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3   0.5  59.24171  68.51439
44  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   2  0.75  57.93839  67.17158
49  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   2  0.25  57.81991  67.15183
48  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   1   0.0  57.81991  67.15183
23  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   2  0.75  57.46445  66.69764
50  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   2   0.5  56.99052  66.32245
27  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   1   0.0  56.75355  66.08548
28  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   2  0.25  56.75355  66.08548
29  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   2   0.5  56.04265  65.37458
52  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   3  0.25  55.92417  65.25610
53  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   3   0.5  54.97630  64.30823
47  indonli_mnli_idkmrc-nli  idkmrc   2   4   2   3  0.75  54.97630  64.23911
31  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   3  0.25  54.62085  63.95278
26  indonli_mnli_idkmrc-nli  idkmrc   1   4   2   3  0.75  54.26540  63.52821
32  indonli_mnli_idkmrc-nli  idkmrc   1   4   3   3   0.5  53.79147  63.12339
2                   indonli  idkmrc   1   4   2   1   0.0  54.62085  62.08946
51  indonli_mnli_idkmrc-nli  idkmrc   2   4   3   2  0.75  52.36967  61.61076
30  indonli_mnli_idkmrc-nli  idkmrc   1   4   

# TYDI-QA-ID

In [10]:
data = "tydiqaid"
url_list = take_all_url(data)
table = pd.DataFrame(create_table(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 55


Processing URLs: 100%|█████████████████████████████████████████████████████████████████| 55/55 [00:46<00:00,  1.18URL/s]


MSC      data  TQ  TS MSI  VA    TH        EM  \
0                    Baseline  tydiqaid  BL  BL  BL  BL    BL  61.28655   
1                     indonli  tydiqaid   1   4   1   1   0.0  35.90643   
2                     indonli  tydiqaid   1   4   2   1   0.0  36.14035   
3                     indonli  tydiqaid   1   4   3   1   0.0  36.25731   
4                     indonli  tydiqaid   2   4   1   1   0.0  59.76608   
5                     indonli  tydiqaid   2   4   2   1   0.0  60.00000   
6                     indonli  tydiqaid   2   4   3   1   0.0  60.11696   
7                indonli_mnli  tydiqaid   1   4   1   1   0.0  42.92398   
8                indonli_mnli  tydiqaid   1   4   2   1   0.0  43.50877   
9                indonli_mnli  tydiqaid   1   4   3   1   0.0  43.50877   
10               indonli_mnli  tydiqaid   2   4   1   1   0.0  59.88304   
11               indonli_mnli  tydiqaid   2   4   2   1   0.0  60.00000   
12               indonli_mnli  tydiqaid   2   4   3   1   0.0  60.00000   
13  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   1   0.0  61.16959   
14  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2  0.25  61.16959   
15  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2   0.5  61.16959   
16  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2  0.75  60.70175   
17  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3  0.25  61.28655   
18  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3   0.5  61.28655   
19  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3  0.75  61.28655   
20  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   1   0.0  61.28655   
21  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2  0.25  61.28655   
22  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2   0.5  61.28655   
23  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2  0.75  60.93567   
24  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3  0.25  61.28655   
25  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3   0.5  61.28655   
26  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3  0.75  61.16959   
27  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   1   0.0  61.28655   
28  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2  0.25  61.28655   
29  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2   0.5  61.28655   
30  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2  0.75  60.93567   
31  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3  0.25  61.28655   
32  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3   0.5  61.28655   
33  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3  0.75  61.05263   
34  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   1   0.0  61.16959   
35  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2  0.25  61.16959   
36  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2   0.5  61.16959   
37  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2  0.75  60.70175   
38  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3  0.25  61.28655   
39  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3   0.5  61.28655   
40  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3  0.75  61.28655   
41  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   1   0.0  61.28655   
42  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2  0.25  61.28655   
43  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2   0.5  61.28655   
44  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2  0.75  60.93567   
45  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3  0.25  61.28655   
46  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3   0.5  61.28655   
47  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3  0.75  61.16959   
48  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   1   0.0  61.28655   
49  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2  0.25  61.28655   
50  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2   0.5  61.28655   
51  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2  0.75  60.93567   
52  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3

In [13]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

tydiqaid sorted by EM


MSC      data  TQ  TS MSI  VA    TH        EM  \
0                    Baseline  tydiqaid  BL  BL  BL  BL    BL  61.28655   
41  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   1   0.0  61.28655   
25  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3   0.5  61.28655   
28  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2  0.25  61.28655   
29  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2   0.5  61.28655   
31  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3  0.25  61.28655   
32  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3   0.5  61.28655   
38  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3  0.25  61.28655   
39  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3   0.5  61.28655   
40  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3  0.75  61.28655   
42  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2  0.25  61.28655   
22  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2   0.5  61.28655   
43  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2   0.5  61.28655   
45  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3  0.25  61.28655   
46  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3   0.5  61.28655   
48  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   1   0.0  61.28655   
49  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2  0.25  61.28655   
50  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2   0.5  61.28655   
52  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   3  0.25  61.28655   
53  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   3   0.5  61.28655   
24  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3  0.25  61.28655   
27  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   1   0.0  61.28655   
21  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2  0.25  61.28655   
20  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   1   0.0  61.28655   
19  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3  0.75  61.28655   
18  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3   0.5  61.28655   
17  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3  0.25  61.28655   
34  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   1   0.0  61.16959   
47  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3  0.75  61.16959   
13  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   1   0.0  61.16959   
36  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2   0.5  61.16959   
35  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2  0.25  61.16959   
14  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2  0.25  61.16959   
15  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2   0.5  61.16959   
26  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3  0.75  61.16959   
33  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3  0.75  61.05263   
54  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   3  0.75  61.05263   
44  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2  0.75  60.93567   
30  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2  0.75  60.93567   
51  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2  0.75  60.93567   
23  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2  0.75  60.93567   
16  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2  0.75  60.70175   
37  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2  0.75  60.70175   
6                     indonli  tydiqaid   2   4   3   1   0.0  60.11696   
11               indonli_mnli  tydiqaid   2   4   2   1   0.0  60.00000   
12               indonli_mnli  tydiqaid   2   4   3   1   0.0  60.00000   
5                     indonli  tydiqaid   2   4   2   1   0.0  60.00000   
10               indonli_mnli  tydiqaid   2   4   1   1   0.0  59.88304   
4                     indonli  tydiqaid   2   4   1   1   0.0  59.76608   
9                indonli_mnli  tydiqaid   1   4   3   1   0.0  43.50877   
8                indonli_mnli  tydiqaid   1   4   2   1   0.0  43.50877   
7                indonli_mnli  tydiqaid   1   4   1   1   0.0  42.92398   
3                     indonli  tydiqaid   1   4   3

In [14]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

tydiqaid sorted by F1


MSC      data  TQ  TS MSI  VA    TH        EM  \
25  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3   0.5  61.28655   
46  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3   0.5  61.28655   
45  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3  0.25  61.28655   
24  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3  0.25  61.28655   
47  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   3  0.75  61.16959   
26  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   3  0.75  61.16959   
31  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3  0.25  61.28655   
53  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   3   0.5  61.28655   
52  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   3  0.25  61.28655   
32  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3   0.5  61.28655   
27  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   1   0.0  61.28655   
21  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2  0.25  61.28655   
20  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   1   0.0  61.28655   
28  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2  0.25  61.28655   
29  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2   0.5  61.28655   
42  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2  0.25  61.28655   
41  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   1   0.0  61.28655   
43  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2   0.5  61.28655   
48  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   1   0.0  61.28655   
49  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2  0.25  61.28655   
50  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2   0.5  61.28655   
22  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2   0.5  61.28655   
33  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   3  0.75  61.05263   
54  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   3  0.75  61.05263   
51  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   3   2  0.75  60.93567   
30  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   3   2  0.75  60.93567   
23  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   2   2  0.75  60.93567   
44  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   2   2  0.75  60.93567   
38  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3  0.25  61.28655   
40  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3  0.75  61.28655   
19  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3  0.75  61.28655   
39  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   3   0.5  61.28655   
0                    Baseline  tydiqaid  BL  BL  BL  BL    BL  61.28655   
18  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3   0.5  61.28655   
17  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   3  0.25  61.28655   
15  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2   0.5  61.16959   
13  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   1   0.0  61.16959   
14  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2  0.25  61.16959   
36  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2   0.5  61.16959   
35  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2  0.25  61.16959   
34  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   1   0.0  61.16959   
37  indonli_mnli_tydiqaid-nli  tydiqaid   2   4   1   2  0.75  60.70175   
16  indonli_mnli_tydiqaid-nli  tydiqaid   1   4   1   2  0.75  60.70175   
6                     indonli  tydiqaid   2   4   3   1   0.0  60.11696   
12               indonli_mnli  tydiqaid   2   4   3   1   0.0  60.00000   
11               indonli_mnli  tydiqaid   2   4   2   1   0.0  60.00000   
5                     indonli  tydiqaid   2   4   2   1   0.0  60.00000   
10               indonli_mnli  tydiqaid   2   4   1   1   0.0  59.88304   
4                     indonli  tydiqaid   2   4   1   1   0.0  59.76608   
9                indonli_mnli  tydiqaid   1   4   3   1   0.0  43.50877   
8                indonli_mnli  tydiqaid   1   4   2   1   0.0  43.50877   
7                indonli_mnli  tydiqaid   1   4   1   1   0.0  42.92398   
3                     indonli  tydiqaid   1   4   3

# SQUAD-ID

In [16]:
data = "squadid"
url_list = take_all_url(data)
table = pd.DataFrame(create_table(url_list, data))
table.to_excel(f'{data}-nli-validator.xlsx', index=False)
table

Amount of url: 55


Processing URLs: 100%|█████████████████████████████████████████████████████████████████| 55/55 [00:46<00:00,  1.18URL/s]


MSC     data  TQ  TS MSI  VA    TH        EM  \
0                   Baseline  squadid  BL  BL  BL  BL    BL  47.91544   
1                    indonli  squadid   1   4   1   1   0.0  13.66967   
2                    indonli  squadid   1   4   2   1   0.0  13.78759   
3                    indonli  squadid   1   4   3   1   0.0  13.79601   
4                    indonli  squadid   2   4   1   1   0.0  40.84056   
5                    indonli  squadid   2   4   2   1   0.0  40.95848   
6                    indonli  squadid   2   4   3   1   0.0  40.95848   
7               indonli_mnli  squadid   1   4   1   1   0.0  21.88158   
8               indonli_mnli  squadid   1   4   2   1   0.0  22.02476   
9               indonli_mnli  squadid   1   4   3   1   0.0  22.05845   
10              indonli_mnli  squadid   2   4   1   1   0.0  41.75019   
11              indonli_mnli  squadid   2   4   2   1   0.0  41.82599   
12              indonli_mnli  squadid   2   4   3   1   0.0  41.82599   
13  indonli_mnli_squadid-nli  squadid   1   4   1   1   0.0  47.64592   
14  indonli_mnli_squadid-nli  squadid   1   4   1   2  0.25  47.64592   
15  indonli_mnli_squadid-nli  squadid   1   4   1   2   0.5  47.64592   
16  indonli_mnli_squadid-nli  squadid   1   4   1   2  0.75  47.24164   
17  indonli_mnli_squadid-nli  squadid   1   4   1   3  0.25  47.91544   
18  indonli_mnli_squadid-nli  squadid   1   4   1   3   0.5  47.91544   
19  indonli_mnli_squadid-nli  squadid   1   4   1   3  0.75  47.91544   
20  indonli_mnli_squadid-nli  squadid   1   4   2   1   0.0  47.76383   
21  indonli_mnli_squadid-nli  squadid   1   4   2   2  0.25  47.76383   
22  indonli_mnli_squadid-nli  squadid   1   4   2   2   0.5  47.76383   
23  indonli_mnli_squadid-nli  squadid   1   4   2   2  0.75  47.50274   
24  indonli_mnli_squadid-nli  squadid   1   4   2   3  0.25  47.80595   
25  indonli_mnli_squadid-nli  squadid   1   4   2   3   0.5  47.80595   
26  indonli_mnli_squadid-nli  squadid   1   4   2   3  0.75  47.67119   
27  indonli_mnli_squadid-nli  squadid   1   4   3   1   0.0  47.76383   
28  indonli_mnli_squadid-nli  squadid   1   4   3   2  0.25  47.76383   
29  indonli_mnli_squadid-nli  squadid   1   4   3   2   0.5  47.76383   
30  indonli_mnli_squadid-nli  squadid   1   4   3   2  0.75  47.51116   
31  indonli_mnli_squadid-nli  squadid   1   4   3   3  0.25  47.78068   
32  indonli_mnli_squadid-nli  squadid   1   4   3   3   0.5  47.78068   
33  indonli_mnli_squadid-nli  squadid   1   4   3   3  0.75  47.55327   
34  indonli_mnli_squadid-nli  squadid   2   4   1   1   0.0  47.64592   
35  indonli_mnli_squadid-nli  squadid   2   4   1   2  0.25  47.64592   
36  indonli_mnli_squadid-nli  squadid   2   4   1   2   0.5  47.64592   
37  indonli_mnli_squadid-nli  squadid   2   4   1   2  0.75  47.24164   
38  indonli_mnli_squadid-nli  squadid   2   4   1   3  0.25  47.91544   
39  indonli_mnli_squadid-nli  squadid   2   4   1   3   0.5  47.91544   
40  indonli_mnli_squadid-nli  squadid   2   4   1   3  0.75  47.91544   
41  indonli_mnli_squadid-nli  squadid   2   4   2   1   0.0  47.76383   
42  indonli_mnli_squadid-nli  squadid   2   4   2   2  0.25  47.76383   
43  indonli_mnli_squadid-nli  squadid   2   4   2   2   0.5  47.76383   
44  indonli_mnli_squadid-nli  squadid   2   4   2   2  0.75  47.50274   
45  indonli_mnli_squadid-nli  squadid   2   4   2   3  0.25  47.80595   
46  indonli_mnli_squadid-nli  squadid   2   4   2   3   0.5  47.80595   
47  indonli_mnli_squadid-nli  squadid   2   4   2   3  0.75  47.67119   
48  indonli_mnli_squadid-nli  squadid   2   4   3   1   0.0  47.76383   
49  indonli_mnli_squadid-nli  squadid   2   4   3   2  0.25  47.76383   
50  indonli_mnli_squadid-nli  squadid   2   4   3   2   0.5  47.76383   
51  indonli_mnli_squadid-nli  squadid   2   4   3   2  0.75  47.51116   
52  indonli_mnli_squadid-nli  squadid   2   4   3   3  0.25  47.78068   
53  indonli_mnli_squadid-nli  squadid   2   4   3   3   0.5  47.78068   
54  indonl

In [17]:
print(f"{data} sorted by EM")
sorted_table = table.sort_values(by='EM', ascending=False)
sorted_table

squadid sorted by EM


MSC     data  TQ  TS MSI  VA    TH        EM  \
0                   Baseline  squadid  BL  BL  BL  BL    BL  47.91544   
19  indonli_mnli_squadid-nli  squadid   1   4   1   3  0.75  47.91544   
18  indonli_mnli_squadid-nli  squadid   1   4   1   3   0.5  47.91544   
17  indonli_mnli_squadid-nli  squadid   1   4   1   3  0.25  47.91544   
38  indonli_mnli_squadid-nli  squadid   2   4   1   3  0.25  47.91544   
39  indonli_mnli_squadid-nli  squadid   2   4   1   3   0.5  47.91544   
40  indonli_mnli_squadid-nli  squadid   2   4   1   3  0.75  47.91544   
24  indonli_mnli_squadid-nli  squadid   1   4   2   3  0.25  47.80595   
25  indonli_mnli_squadid-nli  squadid   1   4   2   3   0.5  47.80595   
45  indonli_mnli_squadid-nli  squadid   2   4   2   3  0.25  47.80595   
46  indonli_mnli_squadid-nli  squadid   2   4   2   3   0.5  47.80595   
32  indonli_mnli_squadid-nli  squadid   1   4   3   3   0.5  47.78068   
31  indonli_mnli_squadid-nli  squadid   1   4   3   3  0.25  47.78068   
52  indonli_mnli_squadid-nli  squadid   2   4   3   3  0.25  47.78068   
53  indonli_mnli_squadid-nli  squadid   2   4   3   3   0.5  47.78068   
43  indonli_mnli_squadid-nli  squadid   2   4   2   2   0.5  47.76383   
42  indonli_mnli_squadid-nli  squadid   2   4   2   2  0.25  47.76383   
48  indonli_mnli_squadid-nli  squadid   2   4   3   1   0.0  47.76383   
49  indonli_mnli_squadid-nli  squadid   2   4   3   2  0.25  47.76383   
50  indonli_mnli_squadid-nli  squadid   2   4   3   2   0.5  47.76383   
29  indonli_mnli_squadid-nli  squadid   1   4   3   2   0.5  47.76383   
28  indonli_mnli_squadid-nli  squadid   1   4   3   2  0.25  47.76383   
41  indonli_mnli_squadid-nli  squadid   2   4   2   1   0.0  47.76383   
27  indonli_mnli_squadid-nli  squadid   1   4   3   1   0.0  47.76383   
21  indonli_mnli_squadid-nli  squadid   1   4   2   2  0.25  47.76383   
20  indonli_mnli_squadid-nli  squadid   1   4   2   1   0.0  47.76383   
22  indonli_mnli_squadid-nli  squadid   1   4   2   2   0.5  47.76383   
26  indonli_mnli_squadid-nli  squadid   1   4   2   3  0.75  47.67119   
47  indonli_mnli_squadid-nli  squadid   2   4   2   3  0.75  47.67119   
13  indonli_mnli_squadid-nli  squadid   1   4   1   1   0.0  47.64592   
35  indonli_mnli_squadid-nli  squadid   2   4   1   2  0.25  47.64592   
14  indonli_mnli_squadid-nli  squadid   1   4   1   2  0.25  47.64592   
15  indonli_mnli_squadid-nli  squadid   1   4   1   2   0.5  47.64592   
36  indonli_mnli_squadid-nli  squadid   2   4   1   2   0.5  47.64592   
34  indonli_mnli_squadid-nli  squadid   2   4   1   1   0.0  47.64592   
54  indonli_mnli_squadid-nli  squadid   2   4   3   3  0.75  47.55327   
33  indonli_mnli_squadid-nli  squadid   1   4   3   3  0.75  47.55327   
51  indonli_mnli_squadid-nli  squadid   2   4   3   2  0.75  47.51116   
30  indonli_mnli_squadid-nli  squadid   1   4   3   2  0.75  47.51116   
44  indonli_mnli_squadid-nli  squadid   2   4   2   2  0.75  47.50274   
23  indonli_mnli_squadid-nli  squadid   1   4   2   2  0.75  47.50274   
37  indonli_mnli_squadid-nli  squadid   2   4   1   2  0.75  47.24164   
16  indonli_mnli_squadid-nli  squadid   1   4   1   2  0.75  47.24164   
11              indonli_mnli  squadid   2   4   2   1   0.0  41.82599   
12              indonli_mnli  squadid   2   4   3   1   0.0  41.82599   
10              indonli_mnli  squadid   2   4   1   1   0.0  41.75019   
6                    indonli  squadid   2   4   3   1   0.0  40.95848   
5                    indonli  squadid   2   4   2   1   0.0  40.95848   
4                    indonli  squadid   2   4   1   1   0.0  40.84056   
9               indonli_mnli  squadid   1   4   3   1   0.0  22.05845   
8               indonli_mnli  squadid   1   4   2   1   0.0  22.02476   
7               indonli_mnli  squadid   1   4   1   1   0.0  21.88158   
3                    indonli  squadid   1   4   3   1   0.0  13.79601   
2                    indonli  squadid   1   4   2   1   0.0  13.78759   
1         

In [18]:
print(f"{data} sorted by F1")
sorted_table = table.sort_values(by='F1', ascending=False)
sorted_table

squadid sorted by F1


MSC     data  TQ  TS MSI  VA    TH        EM  \
0                   Baseline  squadid  BL  BL  BL  BL    BL  47.91544   
40  indonli_mnli_squadid-nli  squadid   2   4   1   3  0.75  47.91544   
19  indonli_mnli_squadid-nli  squadid   1   4   1   3  0.75  47.91544   
17  indonli_mnli_squadid-nli  squadid   1   4   1   3  0.25  47.91544   
39  indonli_mnli_squadid-nli  squadid   2   4   1   3   0.5  47.91544   
38  indonli_mnli_squadid-nli  squadid   2   4   1   3  0.25  47.91544   
18  indonli_mnli_squadid-nli  squadid   1   4   1   3   0.5  47.91544   
45  indonli_mnli_squadid-nli  squadid   2   4   2   3  0.25  47.80595   
46  indonli_mnli_squadid-nli  squadid   2   4   2   3   0.5  47.80595   
25  indonli_mnli_squadid-nli  squadid   1   4   2   3   0.5  47.80595   
24  indonli_mnli_squadid-nli  squadid   1   4   2   3  0.25  47.80595   
32  indonli_mnli_squadid-nli  squadid   1   4   3   3   0.5  47.78068   
31  indonli_mnli_squadid-nli  squadid   1   4   3   3  0.25  47.78068   
52  indonli_mnli_squadid-nli  squadid   2   4   3   3  0.25  47.78068   
53  indonli_mnli_squadid-nli  squadid   2   4   3   3   0.5  47.78068   
29  indonli_mnli_squadid-nli  squadid   1   4   3   2   0.5  47.76383   
28  indonli_mnli_squadid-nli  squadid   1   4   3   2  0.25  47.76383   
27  indonli_mnli_squadid-nli  squadid   1   4   3   1   0.0  47.76383   
49  indonli_mnli_squadid-nli  squadid   2   4   3   2  0.25  47.76383   
48  indonli_mnli_squadid-nli  squadid   2   4   3   1   0.0  47.76383   
50  indonli_mnli_squadid-nli  squadid   2   4   3   2   0.5  47.76383   
42  indonli_mnli_squadid-nli  squadid   2   4   2   2  0.25  47.76383   
43  indonli_mnli_squadid-nli  squadid   2   4   2   2   0.5  47.76383   
41  indonli_mnli_squadid-nli  squadid   2   4   2   1   0.0  47.76383   
20  indonli_mnli_squadid-nli  squadid   1   4   2   1   0.0  47.76383   
21  indonli_mnli_squadid-nli  squadid   1   4   2   2  0.25  47.76383   
22  indonli_mnli_squadid-nli  squadid   1   4   2   2   0.5  47.76383   
26  indonli_mnli_squadid-nli  squadid   1   4   2   3  0.75  47.67119   
47  indonli_mnli_squadid-nli  squadid   2   4   2   3  0.75  47.67119   
33  indonli_mnli_squadid-nli  squadid   1   4   3   3  0.75  47.55327   
54  indonli_mnli_squadid-nli  squadid   2   4   3   3  0.75  47.55327   
30  indonli_mnli_squadid-nli  squadid   1   4   3   2  0.75  47.51116   
51  indonli_mnli_squadid-nli  squadid   2   4   3   2  0.75  47.51116   
23  indonli_mnli_squadid-nli  squadid   1   4   2   2  0.75  47.50274   
44  indonli_mnli_squadid-nli  squadid   2   4   2   2  0.75  47.50274   
36  indonli_mnli_squadid-nli  squadid   2   4   1   2   0.5  47.64592   
34  indonli_mnli_squadid-nli  squadid   2   4   1   1   0.0  47.64592   
35  indonli_mnli_squadid-nli  squadid   2   4   1   2  0.25  47.64592   
15  indonli_mnli_squadid-nli  squadid   1   4   1   2   0.5  47.64592   
14  indonli_mnli_squadid-nli  squadid   1   4   1   2  0.25  47.64592   
13  indonli_mnli_squadid-nli  squadid   1   4   1   1   0.0  47.64592   
37  indonli_mnli_squadid-nli  squadid   2   4   1   2  0.75  47.24164   
16  indonli_mnli_squadid-nli  squadid   1   4   1   2  0.75  47.24164   
12              indonli_mnli  squadid   2   4   3   1   0.0  41.82599   
11              indonli_mnli  squadid   2   4   2   1   0.0  41.82599   
10              indonli_mnli  squadid   2   4   1   1   0.0  41.75019   
6                    indonli  squadid   2   4   3   1   0.0  40.95848   
5                    indonli  squadid   2   4   2   1   0.0  40.95848   
4                    indonli  squadid   2   4   1   1   0.0  40.84056   
9               indonli_mnli  squadid   1   4   3   1   0.0  22.05845   
8               indonli_mnli  squadid   1   4   2   1   0.0  22.02476   
7               indonli_mnli  squadid   1   4   1   1   0.0  21.88158   
3                    indonli  squadid   1   4   3   1   0.0  13.79601   
2                    indonli  squadid   1   4   2   1   0.0  13.78759   
1         